# Homework 3 Part 2

## Mohammed Oubia

## Optimization via Stochastic Gradient Descent


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

• Given a number Nbatch (usually called batch size), randomly extract a subdataset M with |M| =
Nbatch from D.

In [2]:
def GD(f, grad_f, x0, D, kmax,alpha):
    X, y = D
    N = X.shape[1]
    Xhat = np.concatenate((np.ones((1,N)), X), axis=0)
    xsol=x0
    x=[x0]
    i=0
    cond = True
    while i < kmax :
        x.append(xsol)
        i+=1
    return x
def sgd_optimizer(loss, grad_loss, w0, D, alpha, batch_size, n_epochs):
    X, y = D # Unpack the data\n",
    N = X.shape[1]
    d = w0.shape[0]
    idx = np.arange(0, N)
 
 
    # Initialization of history vectors\n",
    w_history = np.zeros((n_epochs, d)) # Save weights at each iteration\n",
    loss_history = np.zeros((n_epochs, )) # Save loss values at each iteration\n",
    grad_norm_history = np.zeros((n_epochs, )) # Save gradient norms at each iteration\n",

    # Initialize weights\n",
    w = w0 
    for epoch in range(n_epochs):
        # Shuffle the data at the beginning of each epoch
        np.random.shuffle(idx)
        X = X[:,idx]
        y = y[idx]

        # Initialize a vector that saves the gradient of the loss at each iteration\n",
        grad_loss_vec = []

        for batch_start in range(0, N, batch_size):
            batch_end = min(batch_start + batch_size, N)
            X_batch = X[:,batch_start:batch_end]
            y_batch = y[batch_start:batch_end]
            # Compute the gradient of the loss
            gradient = grad_loss(w, X_batch, y_batch)

            grad_loss_vec.append(np.linalg.norm(gradient, 2))

            # Update weights
            w = w - alpha* gradient

        # Save the updated values
        w_history[epoch] = w[:,0]
        loss_history[epoch] = loss(w, X, y)
        grad_norm_history[epoch] = np.mean(grad_loss_vec)

    return w_history, loss_history, grad_norm_history
 
 
 
def loss(f, w, D):
    X, y = D
    y_pred = f(w, X)
    return np.mean(np.square(y_pred -y))
def grad_l(f, grad_f, w, D):
    X, y = D
    return np.mean(2 *grad_f(w, X).T * (f(w, X) - y))

• To test the script above, consider the MNIST dataset we used in the previous laboratories, and do the
following:

In [4]:
data = loadmat('MNIST.mat')
X=data["X"]
Y=data["I"]
print(X.shape,Y.shape)

(256, 1707) (1, 1707)


1. From the dataset, select only two digits. It would be great to let the user input the two digits to
select.

In [6]:
C1=int(input())
C2=int(input())
idx=(Y[0,:]==C1 )|( Y[0,:]==C2)
X=X[:,idx]
Y=Y[:,idx]
for i in range(Y.shape[1]):
    if Y[:,i]==C1:
        Y[:,i]=0
    else:
        Y[:,i]=1
d,N=X.shape
Y = Y.reshape((N, 1))
print(X.shape)
print(Y.shape)

2
3
(256, 333)
(333, 1)


2. Do the same operation of the previous homework to obtain the training and test set from (X, Y ),
selecting the Ntrain you prefer.

In [7]:
def split_data(X, Y, Ntrain):
    d, N = X.shape
    idx = np.arange(N)
    np.random.shuffle(idx)
    train_idx = idx[:Ntrain]
    test_idx = idx[Ntrain:]
    Xtrain = X[:, train_idx]
    Ytrain = Y[train_idx]
    Xtest = X[:, test_idx]
    Ytest = Y[test_idx]
    return (Xtrain, Ytrain), (Xtest, Ytest)


In [8]:
(Xtrain, Ytrain), (Xtest, Ytest) = split_data(X, Y, 180)
print(Xtrain.shape, Xtest.shape,Ytrain.shape)
d,N=Xtrain.shape
print(d,N)

(256, 180) (256, 153) (180, 1)
256 180


In [9]:
d,N=Xtrain.shape
print(d,N)
XTrainhat = np.concatenate((np.ones((N,1)).T, Xtrain), axis=0)

256 180


In [10]:
print(XTrainhat.shape)
print(XTrainhat)

(257, 180)
[[1.     1.     1.     ... 1.     1.     1.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.059  0.    ]
 ...
 [0.     0.     0.     ... 0.3175 0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]]


3. Consider Lecture 11 on my website about the implementation of Logistic Regression for binary
classification https://www.evangelistadavide.com/teaching/logistic_regression2024/. It
has not been described in class, but it is equivalent to Linear Regression with a slightly modified
model. After carefully reading the post, implement a logistic regression classificator.

In [14]:
# Define sigmoid
def sigmoid(x):
    return 1/(1+np.exp(-x))
 
# Compute the value of f
def f(w, xhat):
    d,N=xhat.shape
    out=np.zeros((N,1))
    for i in range(len(out)):
        out[i]=sigmoid(xhat[:,i].T@w)
    return out
 
# Value of the loss
def ell(w, Xhat, Y):
    d,N=Xhat.shape
    return 1/2*N*np.linalg.norm(f(w,Xhat)-Y,2)
# Value of the gradient
def grad_ell(w, Xhat, Y):
    d,N=Xhat.shape
    return (Xhat/N) @(sigmoid(Xhat.T@w) *(1-sigmoid(Xhat.T @ w)) * (f(w,Xhat)-Y))

• Test the logistic regression classificator for different digits and different training set dimensions.


In [15]:
k,N=XTrainhat.shape
w0=np.random.rand(k,1)* 0.01
print((XTrainhat @(sigmoid(XTrainhat.T@w0) *(1-sigmoid(XTrainhat.T @ w0)) * (f(w0,XTrainhat)-Ytrain))).shape)
print(f(w0,XTrainhat).shape)

(257, 1)
(180, 1)


In [16]:
print(np.linalg.norm(grad_ell(w0,XTrainhat,Ytrain)))

0.34491991403046557


In [17]:
w_history, loss_history, grad_norm_history=sgd_optimizer(ell, grad_ell, w0, (XTrainhat,Ytrain), 1e-3, 10, 100)
def accuracy(w, X, Y, treshold=0.5):
    N = X.shape[1]
    a = []
    cnt = 0
    x = np.concatenate((np.ones((1,N)), X), axis=0)
    for i in range(N):
        a.append(1 if f(w,x[:,i:i+1]) > treshold else 0)
        if a[-1] == Y[i,0]:
            cnt += 1
    print("Correct elements guessed:",cnt)
    print("Wrong elements guessed:",N-cnt)
    print("Accuracy of Model:",(cnt/N)*100," %")
    return cnt
def predict(w, X, treshold=0.5):
    a=f(w,X)
    if a>0.5:
        return 1
    else:
        return 0

In [18]:
print(f"with {C1} and {C2} digits the metrics of classifier for traininng are:")
cnt=accuracy(w_history[-1],Xtrain,Ytrain,0.5)
print(f"with {C1} and {C2} digits the metrics of classifier for test are:")
cnt=accuracy(w_history[-1],Xtest,Ytest,0.5)

with 2 and 3 digits the metrics of classifier for traininng are:
Correct elements guessed: 173
Wrong elements guessed: 7
Accuracy of Model: 96.11111111111111  %
with 2 and 3 digits the metrics of classifier for test are:
Correct elements guessed: 145
Wrong elements guessed: 8
Accuracy of Model: 94.77124183006535  %


• The training procedure will end up with a set of optimal parameters w
∗
. Compare w
∗ when computed
with Gradient Descent and Stochastic Gradient Descent, for different digits and different training set
dimensions.


with Gradient Descent

In [19]:
x=GD(ell, grad_ell,w0, (XTrainhat,Ytrain), 1000,1e-4)
print(f"with {C1} and {C2} digits the metrics of classifier for traininng are with GD:")
cnt=accuracy(x[-1],Xtrain,Ytrain,0.5)
print(f"with {C1} and {C2} digits the metrics of classifier for test are with GD:")
cnt=accuracy(x[-1],Xtest,Ytest,0.5)

with 2 and 3 digits the metrics of classifier for traininng are with GD:
Correct elements guessed: 68
Wrong elements guessed: 112
Accuracy of Model: 37.77777777777778  %
with 2 and 3 digits the metrics of classifier for test are with GD:
Correct elements guessed: 63
Wrong elements guessed: 90
Accuracy of Model: 41.17647058823529  %


We try with diffrent numbers

In [23]:
data = loadmat('MNIST.mat')
X=data["X"]
Y=data["I"]
print(X.shape,Y.shape)

(256, 1707) (1, 1707)


In [24]:
C1=int(input())
C2=int(input())
idx=(Y[0,:]==C1 )|( Y[0,:]==C2)
X=X[:,idx]
Y=Y[:,idx]
for i in range(Y.shape[1]):
    if Y[:,i]==C1:
        Y[:,i]=0
    else:
        Y[:,i]=1
d,N=X.shape
Y = Y.reshape((N, 1))
print(X.shape)
print(Y.shape)

6
9
(256, 283)
(283, 1)


In [25]:
(Xtrain, Ytrain), (Xtest, Ytest) = split_data(X, Y, 180)

In [26]:
d,N=Xtrain.shape
print(d,N)
XTrainhat = np.concatenate((np.ones((N,1)).T, Xtrain), axis=0)

256 180


In [27]:
k,N=XTrainhat.shape
w0=np.random.rand(k,1)* 0.01

In [28]:
w_history, loss_history, grad_norm_history=sgd_optimizer(ell, grad_ell, w0, (XTrainhat,Ytrain), 1e-3, 10, 100)

In [29]:
print(f"with {C1} and {C2} digits the metrics of classifier for traininng are:")
cnt=accuracy(w_history[-1],Xtrain,Ytrain,0.5)
print(f"with {C1} and {C2} digits the metrics of classifier for test are:")
cnt=accuracy(w_history[-1],Xtest,Ytest,0.5)

with 6 and 9 digits the metrics of classifier for traininng are:
Correct elements guessed: 179
Wrong elements guessed: 1
Accuracy of Model: 99.44444444444444  %
with 6 and 9 digits the metrics of classifier for test are:
Correct elements guessed: 103
Wrong elements guessed: 0
Accuracy of Model: 100.0  %


## End